# Model training
This use-case is model training.  
By going through this you will know how to use Cascade for metadata tracking, hyperparameter tuning and model selection.  
  
Previous part is the pipeline building and is taken without comments.  
For more detailed description of it see Pipeline building example.

In [1]:
import cascade.data as cdd
import cascade.models as cdm
import cascade.utils as cdu
import cascade.meta as cde

from tqdm import tqdm
import torch
import torchvision
from torchvision.transforms import functional as F
from torch import nn
from sklearn.metrics import accuracy_score

c:\Users\илья\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MNIST_ROOT = 'data'
INPUT_SIZE = 784
BATCH_SIZE = 10

## Defining data pipeline
This part will be without comments 

In [3]:
class NoiseModifier(cdd.Modifier):
    def __getitem__(self, index):
        img, label = self._dataset[index] # get the data from Wrapper, which is _dataset for this Modifier
        img += torch.rand_like(img) * 0.1 # apply random noise with fixed magnitude
        img = torch.clip(img, 0, 255)
        return img, label


train_ds = torchvision.datasets.MNIST(root=MNIST_ROOT,
                                     train=True, 
                                     transform=F.to_tensor,
                                     download=True)
test_ds = torchvision.datasets.MNIST(root=MNIST_ROOT, 
                                    train=False, 
                                    transform=F.to_tensor)

train_ds = cdd.Wrapper(train_ds, 
    meta_prefix={
        'desc': 'This is MNIST dataset of handwritten images, TRAIN PART'
    })

test_ds = cdd.Wrapper(test_ds)

train_ds = NoiseModifier(train_ds)
test_ds = NoiseModifier(test_ds)

train_dl = torch.utils.data.DataLoader(dataset=train_ds, 
                                       batch_size=BATCH_SIZE,
                                       shuffle=True)
test_dl = torch.utils.data.DataLoader(dataset=test_ds,
                                      batch_size=BATCH_SIZE,
                                      shuffle=False)

In [4]:
train_ds.get_meta()

[{'name': 'cascade.data.cyclic_sampler.CyclicSampler',
  'type': 'dataset',
  'len': 1000},
 {'name': '__main__.NoiseModifier', 'type': 'dataset', 'len': 60000},
 {'name': 'cascade.data.dataset.Wrapper',
  'desc': 'This is MNIST dataset of handwritten images, TRAIN PART',
  'type': 'dataset',
  'len': 60000,
  'obj_type': torchvision.datasets.mnist.MNIST}]

## Model definition
Before training we need to define our model. We need regular nn.Module and Cascade's wrapper around it.

In [5]:
# Module defined without any specific changes, except *args and **kwargs in __init__
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, *args, **kwargs):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, y):
         out = self.l1(y)
         out = self.relu(out)
         out = self.l2(out)

         return out


class Classifier(cdu.TorchModel):
    def fit(self, train_dl, num_epochs, lr, *args, **kwargs):
        criterion = nn.CrossEntropyLoss()
        optim = torch.optim.Adam(self._model.parameters(), lr=lr)

        ds_size = len(train_dl)
        for epoch in range(num_epochs):
            for x, (imgs, labels) in enumerate(train_dl): 
                imgs = imgs.reshape(-1, self._model.input_size)

                out = self._model(imgs)
                loss = criterion(out, labels)

                optim.zero_grad()
                loss.backward()
                optim.step() 

                if x % 500 == 0:
                    print (f'Epochs [{epoch}/{num_epochs}], Step[{x}/{ds_size}], Loss: {loss.item():.4f}')

    def evaluate(self, test_dl, metrics_dict, *args, **kwargs):
        pred = []
        gt = []
        for imgs, labels in tqdm(test_dl): 
            imgs = imgs.reshape(-1, self._model.input_size)
            out = torch.argmax(self._model(imgs, *args, **kwargs), -1)

            pred.append(out)
            gt.append(labels)

        pred = torch.concat(pred).detach().numpy()
        gt = torch.concat(gt).detach().numpy()

        for metric_name in metrics_dict:
            self.metrics[metric_name] = metrics_dict[metric_name](gt, pred)

In [6]:
NUM_EPOCHS = 2
LR = 1e-3

model = Classifier(SimpleNN, 
    input_size=INPUT_SIZE,
    hidden_size=100,
    num_classes=10,
    num_epochs=NUM_EPOCHS,
    lr=LR,
    bs=BATCH_SIZE)
model.fit(train_dl, NUM_EPOCHS, LR)

Epochs [0/2], Step[0/100], Loss: 2.3050
Epochs [1/2], Step[0/100], Loss: 1.0621


In [7]:
model.evaluate(test_dl, {'acc': accuracy_score})

100%|██████████| 100/100 [00:01<00:00, 70.90it/s]


In [8]:
model.get_meta()

[{'name': '<__main__.Classifier object at 0x0000028B8AEF3CA0>',
  'created_at': DateTime(2022, 7, 26, 20, 12, 52, 409974, tzinfo=Timezone('UTC')),
  'metrics': {'acc': 0.799},
  'params': {'input_size': 784,
   'hidden_size': 100,
   'num_classes': 10,
   'num_epochs': 2,
   'lr': 0.001,
   'bs': 10},
  'type': 'model'}]

In [9]:
repo = cdm.ModelRepo('./repo')

In [10]:
repo.add_line('linear_nn', Classifier)

ModelLine of 0 models of <class '__main__.Classifier'>

In [11]:
model.update_meta({'train_data': train_ds.get_meta()})
model.get_meta()

[{'name': '<__main__.Classifier object at 0x0000028B8AEF3CA0>',
  'train_data': [{'name': 'cascade.data.cyclic_sampler.CyclicSampler',
    'type': 'dataset',
    'len': 1000},
   {'name': '__main__.NoiseModifier', 'type': 'dataset', 'len': 60000},
   {'name': 'cascade.data.dataset.Wrapper',
    'desc': 'This is MNIST dataset of handwritten images, TRAIN PART',
    'type': 'dataset',
    'len': 60000,
    'obj_type': torchvision.datasets.mnist.MNIST}],
  'created_at': DateTime(2022, 7, 26, 20, 12, 52, 409974, tzinfo=Timezone('UTC')),
  'metrics': {'acc': 0.799},
  'params': {'input_size': 784,
   'hidden_size': 100,
   'num_classes': 10,
   'num_epochs': 2,
   'lr': 0.001,
   'bs': 10},
  'type': 'model'}]

In [12]:
repo['linear_nn'].save(model)

In [13]:
mv = cde.MetricViewer(repo)
mv.plot_table()

## More experiments

In [14]:
params = [
    {'hidden_size': 100,  'num_epochs': 2, 'lr': 0.001, 'bs': 10},
    {'hidden_size': 10,   'num_epochs': 2, 'lr': 0.001, 'bs': 10},
    {'hidden_size': 1000, 'num_epochs': 2, 'lr': 0.001, 'bs': 10}
]

In [15]:
for p in params:
    model = Classifier(SimpleNN,
        **p,
        input_size=INPUT_SIZE,
        num_classes=10)
    model.fit(train_dl, **p)
    model.evaluate(test_dl, {'acc': accuracy_score})
    repo['linear_nn'].save(model)

Epochs [0/2], Step[0/100], Loss: 2.3288
Epochs [1/2], Step[0/100], Loss: 0.7451


100%|██████████| 100/100 [00:01<00:00, 60.02it/s]


Epochs [0/2], Step[0/100], Loss: 2.2649
Epochs [1/2], Step[0/100], Loss: 1.7781


100%|██████████| 100/100 [00:01<00:00, 96.72it/s]


Epochs [0/2], Step[0/100], Loss: 2.3603
Epochs [1/2], Step[0/100], Loss: 0.5784


100%|██████████| 100/100 [00:01<00:00, 55.45it/s]


In [16]:
mv = cde.MetricViewer(repo)
mv.plot_table()